<a href="https://colab.research.google.com/github/GorokhovSemyon/TextClassification/blob/main/Advance_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Загрузка датасета

In [ ]:
from datasets import load_dataset

ds = load_dataset("yelp_review_full")
ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [ ]:
ds['train'][0]

{'label': 4,
 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}

In [ ]:
print(ds['train'][0]['text'])

dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.


In [ ]:
from datasets import DatasetDict

ds = DatasetDict({
    'train': ds['train'].shuffle(seed=1).select(range(5000)),
    'test': ds['test'].shuffle(seed=1).select(range(500)),
})
ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 500
    })
})

In [ ]:
ds['train']

Dataset({
    features: ['label', 'text'],
    num_rows: 500
})

In [ ]:
ds['train'].features

{'label': ClassLabel(names=['1 star', '2 star', '3 stars', '4 stars', '5 stars'], id=None),
 'text': Value(dtype='string', id=None)}

## Токенизация датасета

In [ ]:
from transformers import AutoTokenizer

model_id = 'prajjwal1/bert-tiny'
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer

/Users/21550586/Documents/code/local_work/transformers-tut/.venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


BertTokenizerFast(name_or_path='prajjwal1/bert-tiny', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=128,
    )

tokenized_ds = ds.map(tokenize_function, batched=True)
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 500
    })
})

In [ ]:
tokenized_ds['train'][0]['labels']

1

In [ ]:
len(tokenized_ds['train'][0]['input_ids'])

128

In [ ]:
tokenized_ds['train'][0]['text']

'Last time I visited Duck & Decanter I was very disappointed. I have been visiting this place since high school and it has always been great but for some reason this time everything was off. My sandwich bread was hard and inedible, the topping were sparse, and the pasta salad was just not good.'

## Обучение модели (transformers.Trainer)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=5)
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [ ]:
from transformers import TrainingArguments

training_args  = TrainingArguments(
    output_dir='output',
    learning_rate=1e-5,
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    evaluation_strategy='epoch',
    save_strategy='epoch',
)
training_args

/Users/21550586/Documents/code/local_work/transformers-tut/.venv/lib/python3.9/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_steps=None,
eval_strategy=epoch,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsd

In [ ]:
import evaluate

metric = evaluate.load("accuracy")
metric

EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['test'],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

  0%|          | 0/237 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

{'eval_loss': 1.5937070846557617, 'eval_accuracy': 0.268, 'eval_runtime': 0.6838, 'eval_samples_per_second': 731.249, 'eval_steps_per_second': 11.7, 'epoch': 1.0}


  0%|          | 0/8 [00:00<?, ?it/s]

{'eval_loss': 1.5854699611663818, 'eval_accuracy': 0.29, 'eval_runtime': 0.2223, 'eval_samples_per_second': 2248.781, 'eval_steps_per_second': 35.98, 'epoch': 2.0}


  0%|          | 0/8 [00:00<?, ?it/s]

{'eval_loss': 1.5837302207946777, 'eval_accuracy': 0.294, 'eval_runtime': 0.1828, 'eval_samples_per_second': 2735.022, 'eval_steps_per_second': 43.76, 'epoch': 3.0}
{'train_runtime': 28.1035, 'train_samples_per_second': 533.741, 'train_steps_per_second': 8.433, 'train_loss': 1.5919636271673918, 'epoch': 3.0}


TrainOutput(global_step=237, training_loss=1.5919636271673918, metrics={'train_runtime': 28.1035, 'train_samples_per_second': 533.741, 'train_steps_per_second': 8.433, 'total_flos': 4768784640000.0, 'train_loss': 1.5919636271673918, 'epoch': 3.0})

## PyTorch-пайплайн обучения

In [ ]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 500
    })
})

In [ ]:
import torch
from torch.utils.data import Dataset


# Dataset в PyTorch (в отличие от HuggingFace datasets) - это абстрактный базовый класс,
# поэтому придется сделать подкласс с реализацией нужных методов
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = {}
        self.data['labels'] = torch.tensor(data['label'])
        self.data['input_ids'] = torch.tensor(data['input_ids'])
        self.data['attention_mask'] = torch.tensor(data['attention_mask'])
        self.data['token_type_ids'] = torch.tensor(data['token_type_ids'])

    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, idx):
        return {
            'labels': self.data['labels'][idx],
            'input_ids': self.data['input_ids'][idx],
            'attention_mask': self.data['attention_mask'][idx],
            'token_type_ids': self.data['token_type_ids'][idx],
        }

pt_train_ds = CustomDataset(tokenized_ds['train'])
pt_test_ds = CustomDataset(tokenized_ds['test'])
pt_train_ds, pt_test_ds

(<__main__.CustomDataset at 0x31ff14760>,
 <__main__.CustomDataset at 0x17fd5f670>)

In [ ]:
len(pt_train_ds), len(pt_test_ds)

(5000, 500)

In [ ]:
pt_train_ds[0]

{'labels': tensor(1),
 'input_ids': tensor([  101,  2197,  2051,  1045,  4716,  9457,  1004, 11703, 12956,  2099,
          1045,  2001,  2200,  9364,  1012,  1045,  2031,  2042,  5873,  2023,
          2173,  2144,  2152,  2082,  1998,  2009,  2038,  2467,  2042,  2307,
          2021,  2005,  2070,  3114,  2023,  2051,  2673,  2001,  2125,  1012,
          2026, 11642,  7852,  2001,  2524,  1998,  1999,  2098,  7028,  1010,
          1996, 22286,  2020, 20288,  1010,  1998,  1996, 24857, 16521,  2001,
          2074,  2025,  2204,  1012,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,  

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(pt_train_ds, shuffle=True, batch_size=64)
eval_dataloader = DataLoader(pt_test_ds, shuffle=False, batch_size=64)
train_dataloader

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=5)
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=1e-5)
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 1e-05
    maximize: False
    weight_decay: 0.01
)

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    'constant',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
len(train_dataloader)

79

In [ ]:
79 * 64

5056

In [ ]:
num_training_steps

237

In [ ]:
import torch

device = torch.device("mps")  # 'cuda' if available
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [ ]:
from tqdm import tqdm

progress_bar = tqdm(range(num_training_steps))
metric = evaluate.load("accuracy")

for epoch in range(1, num_epochs + 1):

    model.train()

    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)  # forward pass
        loss = outputs.loss
        loss.backward()  # backpropagation
        optimizer.step()  # update weights
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    model.eval()

    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch['labels'])

    print(f'epoch {epoch}/{num_epochs} - {metric.compute()}')


 34%|███▍      | 81/237 [00:08<00:14, 10.77it/s]

epoch 1/3 - {'accuracy': 0.198}


 68%|██████▊   | 161/237 [00:15<00:06, 11.30it/s]

epoch 2/3 - {'accuracy': 0.218}


100%|██████████| 237/237 [00:21<00:00, 12.04it/s]

epoch 3/3 - {'accuracy': 0.258}


100%|██████████| 237/237 [00:34<00:00, 12.04it/s]

## Использование другой модели

In [ ]:
!pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org datasets evaluate transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 37.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7

In [ ]:
from datasets import load_dataset
from datasets import DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np

# Загрузка датасета
ds = load_dataset("yelp_review_full")

# Сокращение датасета для демонстрационных целей
ds = DatasetDict({
    'train': ds['train'].shuffle(seed=1).select(range(5000)),
    'test': ds['test'].shuffle(seed=1).select(range(500)),
})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# Задание модели и токенизатора
model_id = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Функция токенизации
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=128,
    )

# Применение токенизации к датасету
tokenized_ds = ds.map(tokenize_function, batched=True)

ds

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 500
    })
})

In [ ]:
# Загрузка модели
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=5)
model

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from transformers import TrainingArguments

# Настройка параметров обучения
training_args  = TrainingArguments(
    output_dir='output',
    learning_rate=1e-5,
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    evaluation_strategy='epoch',
    save_strategy='epoch',
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Загрузка метрики
metric = evaluate.load("accuracy")

In [ ]:
# Вычисление метрик
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Настройка обучения
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['test'],
    compute_metrics=compute_metrics,
)

# Обучение модели
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.063723,0.552000
2,No log,1.052968,0.542000
3,No log,1.044488,0.550000


TrainOutput(global_step=237, training_loss=1.0105942351908623, metrics={'train_runtime': 338.3523, 'train_samples_per_second': 44.332, 'train_steps_per_second': 0.7, 'total_flos': 986693034240000.0, 'train_loss': 1.0105942351908623, 'epoch': 3.0})

## PyTorch-пайплайн обучения с увеличенным количество эпох на той же тестовой выборке

###Дополнительно осуществлён подбор learning rate и batch size для достижения лучшего результата по метрике accuracy

In [ ]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 500
    })
})

In [ ]:
import torch
from torch.utils.data import Dataset


# Dataset в PyTorch (в отличие от HuggingFace datasets) - это абстрактный базовый класс,
# поэтому придется сделать подкласс с реализацией нужных методов
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = {}
        self.data['labels'] = torch.tensor(data['label'])
        self.data['input_ids'] = torch.tensor(data['input_ids'])
        self.data['attention_mask'] = torch.tensor(data['attention_mask'])
        self.data['token_type_ids'] = torch.tensor(data['token_type_ids'])

    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, idx):
        return {
            'labels': self.data['labels'][idx],
            'input_ids': self.data['input_ids'][idx],
            'attention_mask': self.data['attention_mask'][idx],
            'token_type_ids': self.data['token_type_ids'][idx],
        }

pt_train_ds = CustomDataset(tokenized_ds['train'])
pt_test_ds = CustomDataset(tokenized_ds['test'])
pt_train_ds, pt_test_ds

(<__main__.CustomDataset at 0x7c24c80b6f50>,
 <__main__.CustomDataset at 0x7c24c80b75b0>)

In [ ]:
len(pt_train_ds), len(pt_test_ds)

(5000, 500)

In [ ]:
pt_train_ds[0]

{'labels': tensor(1),
 'input_ids': tensor([  101,  2197,  2051,  1045,  4716,  9457,  1004, 11703, 12956,  2099,
          1045,  2001,  2200,  9364,  1012,  1045,  2031,  2042,  5873,  2023,
          2173,  2144,  2152,  2082,  1998,  2009,  2038,  2467,  2042,  2307,
          2021,  2005,  2070,  3114,  2023,  2051,  2673,  2001,  2125,  1012,
          2026, 11642,  7852,  2001,  2524,  1998,  1999,  2098,  7028,  1010,
          1996, 22286,  2020, 20288,  1010,  1998,  1996, 24857, 16521,  2001,
          2074,  2025,  2204,  1012,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,  

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(pt_train_ds, shuffle=True, batch_size=32)
eval_dataloader = DataLoader(pt_test_ds, shuffle=False, batch_size=32)
train_dataloader

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=5)
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=1e-6)
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 1e-06
    maximize: False
    weight_decay: 0.01
)

In [ ]:
from transformers import get_scheduler

num_epochs = 5
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    'constant',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
len(train_dataloader)

157

In [ ]:
num_training_steps

785

In [ ]:
import torch

device = torch.device("cuda")  # 'cuda' if available
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from tqdm import tqdm

progress_bar = tqdm(range(num_training_steps))
metric = evaluate.load("accuracy")

for epoch in range(1, num_epochs + 1):

    model.train()

    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)  # forward pass
        loss = outputs.loss
        loss.backward()  # backpropagation
        optimizer.step()  # update weights
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    model.eval()

    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch['labels'])

    print(f'epoch {epoch}/{num_epochs} - {metric.compute()}')


 20%|██        | 158/785 [01:44<15:35,  1.49s/it]

epoch 1/5 - {'accuracy': 0.244}


 40%|████      | 315/785 [03:28<11:40,  1.49s/it]

epoch 2/5 - {'accuracy': 0.306}


 60%|██████    | 472/785 [05:13<07:47,  1.49s/it]

epoch 3/5 - {'accuracy': 0.37}


 80%|████████  | 629/785 [06:57<03:53,  1.50s/it]

epoch 4/5 - {'accuracy': 0.496}


100%|██████████| 785/785 [08:38<00:00,  1.55it/s]

epoch 5/5 - {'accuracy': 0.52}


### Таблица *результатов* для Transformers пайплайна:

| Learning Rate | Batch Size | Число эпох | Модель | Accuracy |
|---------------|------------|------------|--------|----------|
| 1e-5          | 64         | 3          | prajjwal1/bert-tiny           | 0.29     |
| 1e-5          | 64         | 3          | bert-base-uncased | 0.55     |


#### Использование более совершенной модели дает лучший результат.

---

####По полученным результатам явно видно, что при одинаковых параметрах обучения, имеем существенную разницу в результате в пользу более сложной модели.

### Таблица результатов для PyTorch пайплайна:

| Learning Rate | Batch Size | Число эпох | Accuracy |
|---------------|------------|------------|----------|
| 1e-5          | 64         | 3          | 0.26     |
| 1e-6          | 32         | 5          | 0.52     |


####Регулирование learning rate и batch size для достижения лучшего результата по метрике accuracy

---

####Эксперименты с изменением параметров обучения, такими как learning rate, batch size и количество эпох, показали, что эти параметры имеют значительное влияние на процесс обучения и точность модели. Оптимальный выбор learning rate и batch size позволяет достичь высокой точности предсказаний и улучшить качество модели. Регулирование этих параметров с учетом конкретных особенностей данных и модели позволяет добиться лучших результатов по метрике accuracy.